In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 100

In [2]:
from loadFile import getPathList, fileToDataFrame

# ディレクトリ 内のメールファイルを読み込む
directory_path = "enron/maildir/allen-p/inbox"
file_list, dir_list=getPathList(directory_path)

# テキストファイルをデータフレームに格納する
df = fileToDataFrame(file_list)

In [3]:
from preProcess import toSentences

sent_dict = {'Message-ID':[],
            'Sentence-ID':[],
            'Sentence':[]}

start = time.time()

for i,message in enumerate(df['Message']):
    sentences = toSentences(message)
    sent_dict['Message-ID'].extend([df['Message-ID'][i]] * len(sentences))
    sent_dict['Sentence-ID'].extend(range(len(sentences)))
    sent_dict['Sentence'].extend(sentences)

sentence_df = pd.DataFrame.from_dict(sent_dict,orient='columns')

end = time.time()
print(end-start,'s')

0.17372894287109375 s


In [5]:
# トリプルテーブルを作る

from tripleExtraction import tripleExtraction

triple_dict = {'Message-ID':[],
             'Sentence-ID':[],
             'Triple':[]}

start = time.time()

counter = 0
for i, sentence in enumerate(sentence_df['Sentence']):
    triples = tripleExtraction(sentence)
    triple_dict['Message-ID'].extend([sentence_df['Message-ID'][i]] * len(triples))
    triple_dict['Sentence-ID'].extend([sentence_df['Sentence-ID'][i]] * len(triples))
    triple_dict['Triple'].extend(triples)
    
triple_df = pd.DataFrame.from_dict(triple_dict,orient='columns')
        
end = time.time()
print(end-start,'s')

393.9632821083069 s


In [6]:
output = pd.merge(df[['Path','Message-ID','Subject']],pd.merge(sentence_df,triple_df,on=['Message-ID','Sentence-ID']),on='Message-ID')

In [8]:
output.to_csv('output/allen_p_inbox_triple2.csv')

In [7]:
output

,Path,Message-ID,Subject,Sentence-ID,Sentence,Triple
0,enron/maildir/allen-p/inbox/36.,<22534132.1075858645229.JavaMail.evans@thyme>,"TVA takes lead in sales, generation by utilities",0,"=20 Updated: Oct. 30, 2001 TVA takes lead in sales, generation by utilities Quasi-governmental ...","(2001 TVA Valley Authority ( TVA, takes, lead)"
1,enron/maildir/allen-p/inbox/36.,<22534132.1075858645229.JavaMail.evans@thyme>,"TVA takes lead in sales, generation by utilities",0,"=20 Updated: Oct. 30, 2001 TVA takes lead in sales, generation by utilities Quasi-governmental ...","(), took, the top spot as the utility with the largest electricity sales and generation)"
2,enron/maildir/allen-p/inbox/36.,<22534132.1075858645229.JavaMail.evans@thyme>,"TVA takes lead in sales, generation by utilities",0,"=20 Updated: Oct. 30, 2001 TVA takes lead in sales, generation by utilities Quasi-governmental ...","(a division of ... Mexican electricity reform, put, recession delays sector restructuring Depend..."
3,enron/maildir/allen-p/inbox/36.,<22534132.1075858645229.JavaMail.evans@thyme>,"TVA takes lead in sales, generation by utilities",0,"=20 Updated: Oct. 30, 2001 TVA takes lead in sales, generation by utilities Quasi-governmental ...","(oil increases Utilities slow to buy ASPs, allow, companies to focus Customer service function a..."
4,enron/maildir/allen-p/inbox/36.,<22534132.1075858645229.JavaMail.evans@thyme>,"TVA takes lead in sales, generation by utilities",0,"=20 Updated: Oct. 30, 2001 TVA takes lead in sales, generation by utilities Quasi-governmental ...","(companies, focus, Customer service function a key attraction)"
...,...,...,...,...,...,...
804,enron/maildir/allen-p/inbox/8.,<7059385.1075855377617.JavaMail.evans@thyme>,"PHILLIP, You've been selected to get FREE software from Sega, IBM,\n Disney & may more...",12,"If you would like to be removed from the United Marketing Strategies email announcement list, cl...","(You, need, to copy and paste the link into your browser)click below)"
805,enron/maildir/allen-p/inbox/8.,<7059385.1075855377617.JavaMail.evans@thyme>,"PHILLIP, You've been selected to get FREE software from Sega, IBM,\n Disney & may more...",12,"If you would like to be removed from the United Marketing Strategies email announcement list, cl...","(You, copy, the link)"
806,enron/maildir/allen-p/inbox/8.,<7059385.1075855377617.JavaMail.evans@thyme>,"PHILLIP, You've been selected to get FREE software from Sega, IBM,\n Disney & may more...",12,"If you would like to be removed from the United Marketing Strategies email announcement list, cl...","(You, paste, the link)"
807,enron/maildir/allen-p/inbox/74.,<9144651.1075862166126.JavaMail.evans@thyme>,Enron North America Bid-Week Prices for West Region (11/27/2001),1,"We ask that this confidential information be treated as such, in accordance with applicable laws...","(We, ask, that this confidential information be treated as such , in accordance with applicable ..."
